## Data Cleaning

This notebook has the purpose of: 
- Cleaning and to combine all the reviews into one table.
- Create a wine table, extracted from the the reviews. one wine can be reviewed many times and we need a wine list for each vineyard.

In [1]:
import os
import re
import numpy as np
import pandas as pd
from collections import Counter

raw_spectator_file = "combined_spectator_reviews.csv"
raw_enthusiast_file = "raw_wine_enthusiast_reviews.csv"


In [373]:
spectator_df = pd.read_csv(raw_spectator_file)
spectator_df

,vintage,score,release_price,bottle_name,winery,review_text,Magazine,review_link,region,source,varietal,wine_id,Region
0,2011,91,$40,Cabernet Franc Rogue Valley,Griffin Creek,"Soft, ripe, generous and precise, this open-te...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Franc,379875.0,NaN
1,2011,86,$40,Arsenal Rogue Valley,Quady North,"A snappy red, if modest in scale, with blackbe...","Magazine Issue Feb 02, 2017",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Franc,429948.0,NaN
2,2011,90,$28,Cabernet Sauvignon Rogue Valley,Del Rio,"A vibrant, medium-weight red, with ripe plum a...","Magazine Issue Feb 28, 2014",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Sauvignon & Blends,357733.0,NaN
3,2011,86,$40,Cabernet Sauvignon Rogue Valley,Griffin Creek,"Lean, spicy and firmly tannic, offering billow...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Sauvignon & Blends,379943.0,NaN
4,2011,94,$125,Chardonnay Dundee Hills Récolte Grand Cru,Domaine Serene,"Tangy, with vibrant acidity and expressive, ri...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Dundee Hills']",Wine Spectator,Chardonnay,380170.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,2015,90,$45,Chardonnay Dundee Hills,Knudsen,"Crisp and harmonious, with lemon blossom and a...",Magazine Issue Web Only - 2017,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Dundee Hills']",Wine Spectator,Chardonnay,434673.0,NaN
1234,2015,90,$40,Chardonnay Willamette Valley Reserve,Ponzi,"Graceful and polished, with expressive pear, s...","Magazine Issue Jul 31, 2018",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator,Chardonnay,454785.0,NaN
1235,2015,90,$45,Chardonnay Yamhill-Carlton District Gran Morai...,Raptor Ridge,"Light and polished, showing pretty pear, pinea...",Magazine Issue Web Only - 2017,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt...",Wine Spectator,Chardonnay,427274.0,NaN
1236,2015,90,$60,Chardonnay Yamhill-Carlton District Marsh Esta...,Roco,"Floral and harmonious, with delicate tropical ...",Magazine Issue Web Only - 2018,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt...",Wine Spectator,Chardonnay,447448.0,NaN


In [6]:
enthusiast_df = pd.read_csv(raw_enthusiast_file)
enthusiast_df

,Unnamed: 0,price,rating,bottle_name,region,review_link,source,Price,Designation,Variety,Appellation,Winery,Alcohol,Bottle Size,Category,Date Published,User Avg Rating,Importer
0,0,$13,88,The Great Oregon Wine Company 2021 Rascal Pino...,Oregon Other,https://www.winemag.com/buying-guide/the-great...,Wine Enthusiast,"['$13', 'Buy Now']",['Rascal'],['Pinot Noir'],"['Oregon', ' Oregon Other', ' Oregon', ' US']",['The Great Oregon Wine Company'],['13.3%'],['750 ml'],['Red'],['10/1/2022'],['Not rated yet [Add Your Review]'],NaN
1,1,$19,86,Wine by Joe 2019 Pinot Noir (Oregon),Oregon Other,https://www.winemag.com/buying-guide/wine-by-j...,Wine Enthusiast,"['$19', 'Buy Now']",NaN,['Pinot Noir'],"['Oregon', ' Oregon Other', ' Oregon', ' US']",['Wine by Joe'],['13.5%'],['750 ml'],['Red'],['10/1/2022'],['Not rated yet [Add Your Review]'],NaN
2,2,$15,88,Duck Pond Cellars 2021 Oregon Wine Co. Chardon...,Oregon Other,https://www.winemag.com/buying-guide/duck-pond...,Wine Enthusiast,"['$15', 'Buy Now']",['Oregon Wine Co.'],['Chardonnay'],"['Oregon', ' Oregon Other', ' Oregon', ' US']",['Duck Pond Cellars'],['13.2%'],['750 ml'],['White'],['9/1/2022'],['Not rated yet [Add Your Review]'],NaN
3,3,$75,92,Domaine Drouhin 2019 Laurene Pinot Noir (Dunde...,Willamette Valley,https://www.winemag.com/buying-guide/domaine-d...,Wine Enthusiast,"['$75', 'Buy Now']",['Laurene'],['Pinot Noir'],"['Dundee Hills', ' Willamette Valley', ' Orego...",['Domaine Drouhin'],['13.9%'],['750 ml'],['Red'],['7/1/2022'],['Not rated yet [Add Your Review]'],NaN
4,4,$40,90,Portlandia 2019 Momtazi Vineyard Pinot Noir (W...,Willamette Valley,https://www.winemag.com/buying-guide/portlandi...,Wine Enthusiast,"['$40', 'Buy Now']",['Momtazi Vineyard'],['Pinot Noir'],"['Willamette Valley', ' Willamette Valley', ' ...",['Portlandia'],['13.5%'],['750 ml'],['Red'],['7/1/2022'],['Not rated yet [Add Your Review]'],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,736,$18,90,Ponzi 1998 Arneis (Willamette Valley),Willamette Valley,https://www.winemag.com/buying-guide/ponzi-199...,Wine Enthusiast,"['$18', 'Buy Now']",NaN,"['Arneis', ' Italian White']","['Willamette Valley', ' Willamette Valley', ' ...",['Ponzi'],['N/A'],['500 ml'],['White'],['4/1/2000'],['Not rated yet [Add Your Review]'],NaN
737,737,$12,86,Tyee 1996 Pinot Blanc (Willamette Valley),Willamette Valley,https://www.winemag.com/buying-guide/tyee-1996...,Wine Enthusiast,"['$12', 'Buy Now']",NaN,['Pinot Blanc'],"['Willamette Valley', ' Willamette Valley', ' ...",['Tyee'],['13%'],['750 ml'],['White'],['12/31/1999'],['Not rated yet [Add Your Review]'],NaN
738,738,$35,92,Cameron 1997 Clos Electrique Chardonnay (Willa...,Willamette Valley,https://www.winemag.com/buying-guide/cameron-1...,Wine Enthusiast,"['$35', 'Buy Now']",['Clos Electrique'],['Chardonnay'],"['Willamette Valley', ' Willamette Valley', ' ...",['Cameron'],['12.7%'],['750 ml'],['White'],['11/1/1999'],['Not rated yet [Add Your Review]'],NaN
739,739,$15,90,Torii Mor 1998 Pinot Gris (Yamhill County),Willamette Valley,https://www.winemag.com/buying-guide/torii-mor...,Wine Enthusiast,"['$15', 'Buy Now']",NaN,['Pinot Gris'],"['Yamhill County', ' Willamette Valley', ' Ore...",['Torii Mor'],['N/A'],['750 ml'],['White'],['11/1/1999'],['Not rated yet [Add Your Review]'],NaN


In [3]:
spectator_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1238 entries, 0 to 1237
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   vintage        1238 non-null   int64  
 1   score          1238 non-null   int64  
 2   release_price  1238 non-null   object 
 3   bottle_name    1238 non-null   object 
 4   winery         1238 non-null   object 
 5   review_text    1238 non-null   object 
 6   Magazine       1238 non-null   object 
 7   review_link    1238 non-null   object 
 8   region         865 non-null    object 
 9   source         1238 non-null   object 
 10  varietal       865 non-null    object 
 11  wine_id        865 non-null    float64
 12  Region         373 non-null    object 
dtypes: float64(1), int64(2), object(10)
memory usage: 125.9+ KB


In [4]:
spectator_df["winery"].value_counts()

Domaine Serene       57
Bergström            56
Evening Land         45
Beaux Frères         41
Alexana              38
                     ..
Coelho                1
Ancient Cellars       1
Anam Cara Cellars     1
Et Fille              1
Authentique           1
Name: winery, Length: 196, dtype: int64

In [7]:
enthusiast_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741 entries, 0 to 740
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       741 non-null    int64 
 1   price            739 non-null    object
 2   rating           741 non-null    int64 
 3   bottle_name      741 non-null    object
 4   region           741 non-null    object
 5   review_link      741 non-null    object
 6   source           741 non-null    object
 7   Price            741 non-null    object
 8   Designation      427 non-null    object
 9   Variety          741 non-null    object
 10  Appellation      741 non-null    object
 11  Winery           741 non-null    object
 12  Alcohol          741 non-null    object
 13  Bottle Size      741 non-null    object
 14  Category         741 non-null    object
 15  Date Published   741 non-null    object
 16  User Avg Rating  741 non-null    object
 17  Importer         4 non-null      ob

Starting with the Wine Enthusiast reviews. They seem to be the dirtiest so far

In [416]:
## First task is to get rid of the list appearance of the string formatted data
cleaning_df = enthusiast_df.drop(['Price', "Unnamed: 0", "Importer"], axis=1)
alcohol = cleaning_df['Alcohol'].copy()
alcohol

0      ['13.3%']
1      ['13.5%']
2      ['13.2%']
3      ['13.9%']
4      ['13.5%']
         ...    
736      ['N/A']
737      ['13%']
738    ['12.7%']
739      ['N/A']
740    ['13.2%']
Name: Alcohol, Length: 741, dtype: object

In [417]:
for i in alcohol.index:
    save = re.search(r"\d+\.*\d*%", alcohol[i])
    if save != None:
        alcohol[i] = save.group()
    else:
        alcohol[i] = None
alcohol

0      13.3%
1      13.5%
2      13.2%
3      13.9%
4      13.5%
       ...  
736     None
737      13%
738    12.7%
739     None
740    13.2%
Name: Alcohol, Length: 741, dtype: object

In [418]:
cleaning_df['Alcohol'] = alcohol
size = cleaning_df['Bottle Size'].copy()
size

0      ['750 ml']
1      ['750 ml']
2      ['750 ml']
3      ['750 ml']
4      ['750 ml']
          ...    
736    ['500 ml']
737    ['750 ml']
738    ['750 ml']
739    ['750 ml']
740    ['750 ml']
Name: Bottle Size, Length: 741, dtype: object

In [419]:
for i in size.index:
    keep = re.search(r"(\d+\s*ml)", size[i])
    if keep != None:
        pass
        size[i] = keep.group()
    else:
        keep = re.search(r"(\d+\s*ML)", size[i])
        if keep != None:
            pass
            size[i] = keep.group()
        else: 
            keep = re.search(r"(\d+\.\d*\s*L)", size[i])
            size[i] = keep.group()

size

0      750 ml
1      750 ml
2      750 ml
3      750 ml
4      750 ml
        ...  
736    500 ml
737    750 ml
738    750 ml
739    750 ml
740    750 ml
Name: Bottle Size, Length: 741, dtype: object

In [420]:
cleaning_df['Bottle Size'] = size
winery = cleaning_df['Winery'].copy()
winery

0      ['The Great Oregon Wine Company']
1                        ['Wine by Joe']
2                  ['Duck Pond Cellars']
3                    ['Domaine Drouhin']
4                         ['Portlandia']
                     ...                
736                            ['Ponzi']
737                             ['Tyee']
738                          ['Cameron']
739                        ['Torii Mor']
740               ['WillaKenzie Estate']
Name: Winery, Length: 741, dtype: object

In [421]:
for i in winery.index:
    winery[i] = winery[i][2:-2]
    
winery    

0      The Great Oregon Wine Company
1                        Wine by Joe
2                  Duck Pond Cellars
3                    Domaine Drouhin
4                         Portlandia
                   ...              
736                            Ponzi
737                             Tyee
738                          Cameron
739                        Torii Mor
740               WillaKenzie Estate
Name: Winery, Length: 741, dtype: object

In [422]:
cleaning_df["Winery"] = winery
variety = cleaning_df["Variety"].copy()
variety

0                    ['Pinot Noir']
1                    ['Pinot Noir']
2                    ['Chardonnay']
3                    ['Pinot Noir']
4                    ['Pinot Noir']
                   ...             
736    ['Arneis', ' Italian White']
737                 ['Pinot Blanc']
738                  ['Chardonnay']
739                  ['Pinot Gris']
740                ['Pinot Grigio']
Name: Variety, Length: 741, dtype: object

In [429]:
for i in variety.index:
    grape = variety[i][2:-2].split("', '")
    variety[i] = grape[0]
    
variety

0        Pinot Noir
1        Pinot Noir
2        Chardonnay
3        Pinot Noir
4        Pinot Noir
           ...     
736          Arneis
737     Pinot Blanc
738      Chardonnay
739      Pinot Gris
740    Pinot Grigio
Name: Variety, Length: 741, dtype: object

In [430]:
cleaning_df["Variety"]  = variety
cleaning_df["Variety"]

0        Pinot Noir
1        Pinot Noir
2        Chardonnay
3        Pinot Noir
4        Pinot Noir
           ...     
736          Arneis
737     Pinot Blanc
738      Chardonnay
739      Pinot Gris
740    Pinot Grigio
Name: Variety, Length: 741, dtype: object

In [431]:
designation = cleaning_df['Designation']
for i in designation.index:
    print(designation[i])
    print(cleaning_df['Winery'][i])
    print(cleaning_df['bottle_name'][i])
    print(cleaning_df['Variety'][i])
    print(cleaning_df['Category'][i])

    print("\n", "=================\n", "")
    

['Rascal']
The Great Oregon Wine Company
The Great Oregon Wine Company 2021 Rascal Pinot Noir (Oregon)
Pinot Noir
['Red']

 
nan
Wine by Joe
Wine by Joe 2019 Pinot Noir (Oregon)
Pinot Noir
['Red']

 
['Oregon Wine Co.']
Duck Pond Cellars
Duck Pond Cellars 2021 Oregon Wine Co. Chardonnay (Oregon)
Chardonnay
['White']

 
['Laurene']
Domaine Drouhin
Domaine Drouhin 2019 Laurene Pinot Noir (Dundee Hills)
Pinot Noir
['Red']

 
['Momtazi Vineyard']
Portlandia
Portlandia 2019 Momtazi Vineyard Pinot Noir (Willamette Valley)
Pinot Noir
['Red']

 
['Unabridged']
Matzinger Davies
Matzinger Davies 2018 Unabridged Sauvignon Blanc (Columbia Gorge (OR))
Sauvignon Blanc
['White']

 
['Ridgecrest Vineyard']
Élevée Winegrowers
Élevée Winegrowers 2020 Ridgecrest Vineyard Riesling (Ribbon Ridge)
Riesling
['White']

 
['Holstein Vineyard Blanc de Noir']
Granville
Granville 2017 Holstein Vineyard Blanc de Noir Sparkling (Dundee Hills)
Sparkling Blend
['Sparkling']

 
nan
Ashlyn
Ashlyn 2019 Pinot Noir (Willa

 
nan
Failla
Failla 2016 Pinot Noir (Willamette Valley)
Pinot Noir
['Red']

 
nan
Moffett
Moffett 2016 Pinot Noir (Dundee Hills)
Pinot Noir
['Red']

 
['Björnson Vineyard']
Failla
Failla 2016 Björnson Vineyard Gamay Noir (Eola-Amity Hills)
Gamay Noir
['Red']

 
['Estate Cuvée']
Troon
Troon 2016 Estate Cuvée Côt (Applegate Valley)
Côt
['Red']

 
['Estate Bottled']
Cardwell Hill
Cardwell Hill 2015 Estate Bottled Pinot Noir (Willamette Valley)
Pinot Noir
['Red']

 
['Blue Line']
Pfeiffer
Pfeiffer 2013 Blue Line Pinot Noir (Oregon)
Pinot Noir
['Red']

 
['Firethorn Vineyard']
Adega Northwest
Adega Northwest 2015 Firethorn Vineyard Syrah (Columbia Valley (OR))
Syrah
['Red']

 
['Firethorn Vineyard Trillium Block']
Adega Northwest
Adega Northwest 2015 Firethorn Vineyard Trillium Block Cabernet Sauvignon (Columbia Valley (OR))
Cabernet Sauvignon
['Red']

 
nan
Carlton Cellars
Carlton Cellars 2017 Auxerrois (Yamhill-Carlton)
Auxerrois
['White']

 
['Cygnus Block']
Brittan Vineyards
Brittan Vin

['Estate']
Bethel Heights
Bethel Heights 2014 Estate Chardonnay (Eola-Amity Hills)
Chardonnay
['White']

 
['Reserve']
Lange
Lange 2015 Reserve Pinot Gris (Willamette Valley)
Pinot Gris
['White']

 
["Cali's Cuvée Estate Grown"]
Left Coast Cellars
Left Coast Cellars 2014 Cali's Cuvée Estate Grown Pinot Noir (Willamette Valley)
Pinot Noir
['Red']

 
['Red Red Wine']
Cliff Creek
Cliff Creek 2015 Red Red Wine Red (Southern Oregon)
Red Blends
['Red']

 
nan
King Estate
King Estate 2015 Chardonnay (Oregon)
Chardonnay
['White']

 
['Estate']
Weisinger
Weisinger 2013 Estate Pinot Noir (Rogue Valley)
Pinot Noir
['Red']

 
['Lone Madrone Vineyard']
Remy
Remy 2013 Lone Madrone Vineyard Lagrein (Willamette Valley)
Lagrein
['Red']

 
['Evenstad Reserve']
Domaine Serene
Domaine Serene 2013 Evenstad Reserve Pinot Noir (Willamette Valley)
Pinot Noir
['Red']

 
nan
Abiqua Wind
Abiqua Wind 2015 Pinot Gris (Willamette Valley)
Pinot Gris
['White']

 
['Estate']
Troon
Troon 2013 Estate Tannat (Applegate V

nan
I'M
I'M 2007 Pinot Noir (Willamette Valley)
Pinot Noir
['Red']

 
nan
Kings Ridge
Kings Ridge 2009 Riesling (Oregon)
Riesling
['White']

 
nan
William Hatcher
William Hatcher 2007 Pinot Noir (Willamette Valley)
Pinot Noir
['Red']

 
['Rudolfo Vineyard']
Lumos
Lumos 2009 Rudolfo Vineyard Pinot Gris (Willamette Valley)
Pinot Gris
['White']

 
['99W']
RouteStock
RouteStock 2009 99W Pinot Noir (Willamette Valley)
Pinot Noir
['Red']

 
['Melon de Bourgogne']
Macindoe
Macindoe 2009 Melon de Bourgogne Melon (Yamhill-Carlton)
Melon
['White']

 
nan
Castle Rock
Castle Rock 2009 Pinot Noir (Willamette Valley)
Pinot Noir
['Red']

 
['Celilo Vineyard']
Dowsett Family
Dowsett Family 2009 Celilo Vineyard Gewürztraminer (Columbia Gorge (WA))
Gewürztraminer
['White']

 
['Bryan Creek Vineyard']
Adelsheim
Adelsheim 2008 Bryan Creek Vineyard Pinot Noir (Chehalem Mountains)
Pinot Noir
['Red']

 
['Syrah 74']
Cowhorn
Cowhorn 2008 Syrah 74 Syrah (Applegate Valley)
Syrah
['Red']

 
nan
Ponzi
Ponzi 2009 

['Domaine']
King Estate
King Estate 2001 Domaine Pinot Noir (Oregon)
Pinot Noir
['White']

 
nan
Rex Hill
Rex Hill 2002 Sauvignon Blanc (Oregon)
Sauvignon Blanc
['White']

 
nan
Cristom
Cristom 2003 Viognier (Willamette Valley)
Viognier
['White']

 
['Red Wine']
Velocity
Velocity 2002 Red Wine Red (Rogue Valley)
Red Blends
['Red']

 
nan
Abacela
Abacela 2003 Grenache (Umpqua Valley)
Grenache
['Red']

 
nan
Ridgefield
Ridgefield 2002 Pinot Gris (Red Mountain)
Pinot Gris
['White']

 
nan
Hogue
Hogue 2003 Pinot Grigio (Columbia Valley (WA))
Pinot Grigio
['White']

 
['Southern Oregon Cuvee']
Rex Hill
Rex Hill 2001 Southern Oregon Cuvee Pinot Noir (Oregon)
Pinot Noir
['Red']

 
["Founders' Reserve"]
Willamette Valley Vineyards
Willamette Valley Vineyards 2002 Founders' Reserve Sauvignon Blanc (Oregon)
Sauvignon Blanc
['White']

 
['Amycas White']
Brooks
Brooks 2002 Amycas White White (Willamette Valley)
White Blend
['White']

 
['Estate Grown']
Paschal
Paschal 2001 Estate Grown Chardonnay 

In [432]:
cleaning_df['vintage'] = cleaning_df['bottle_name'].str.extract(r"(\d{4})")
cleaning_df["vintage"]

0      2021
1      2019
2      2021
3      2019
4      2019
       ... 
736    1998
737    1996
738    1997
739    1998
740    1997
Name: vintage, Length: 741, dtype: object

In [433]:
designation = cleaning_df['Designation'].dropna()
for i in designation.index:
    designation[i] = designation[i][2:-2]
    
designation.value_counts()

Estate                    21
Reserve                   16
Shea Vineyard              8
Red Table Wine             5
Momtazi Vineyard           4
                          ..
Missoulan Wash Reserve     1
Hare Vineyard              1
Grenache                   1
Lazy River Vineyard        1
Clos Electrique            1
Name: Designation, Length: 316, dtype: int64

Willamette Valley
- Chehalem Mountains
- Dundee Hills
- Yamhill-Carlton
- Eola-Amity Hills
- McMinnville
- Laurelwood District
- Ribbon Ridge
- Van Duzer Corridor
- Tualatin Hills
- Mount Pisgah, Polk County
- Lower Long Tom    

Rouge Valley    
- Applegate Valley    

Umpqua Valley    
- Elkton Oregon    
- Red Hill Douglas County    

Columbia Gorge     
Columbia Valley     
Walla Walla Valley     
- The Rocks District of Milton-Freewater   
Snake River Valley   

In [434]:
regions = cleaning_df["Appellation"].apply(lambda x: x[2:-2].split("', ' "))
not_oregon = []
ava = []
region = []
province = []
country = []
for i in regions.index:
    if len(regions[i]) != 4:
        #   # "Washington-Oregon" wines are located in Oregon and include Washington grapes in the making
        if "Washington" in regions[i]:
            province.append((i, "Oregon"))
            country.append((i,regions[i][-1]))
        if i == 333: ## Idaho wine that says US and America only
            ava.append((i, "Snake River Valley (ID)" ))
            region.append((i, "Snake River Valley (ID)" ))
            province.append((i,"Idaho"))
            country.append((i, "US"))

    else: 
        if "Oregon" not in regions[i][0]:
            ava.append((i, regions[i][0]))
            region.append((i,regions[i][1]))
            province.append((i,regions[i][2]))
            country.append((i,regions[i][3]))

        else: 
            region.append((i,regions[i][1]))
            province.append((i,regions[i][2]))
            country.append((i,regions[i][3]))

print(ava, '\n==================\n ')
print(region, '\n==================\n ')


[(3, 'Dundee Hills'), (4, 'Willamette Valley'), (5, 'Columbia Gorge (OR)'), (6, 'Ribbon Ridge'), (7, 'Dundee Hills'), (8, 'Willamette Valley'), (9, 'Willamette Valley'), (10, 'Yamhill-Carlton'), (11, 'McMinnville'), (12, 'Willamette Valley'), (13, 'Willamette Valley'), (14, 'Applegate Valley'), (15, 'Willamette Valley'), (17, 'Willamette Valley'), (18, 'Willamette Valley'), (19, 'Willamette Valley'), (22, 'Willamette Valley'), (23, 'Eola-Amity Hills'), (24, 'Eola-Amity Hills'), (25, 'Willamette Valley'), (26, 'Eola-Amity Hills'), (27, 'Willamette Valley'), (29, 'Eola-Amity Hills'), (30, 'Van Duzer Corridor'), (31, 'Willamette Valley'), (32, 'Willamette Valley'), (35, 'Dundee Hills'), (36, 'Dundee Hills'), (37, 'Willamette Valley'), (39, 'Willamette Valley'), (40, 'Umpqua Valley'), (41, 'Rogue Valley'), (42, 'Willamette Valley'), (45, 'Eola-Amity Hills'), (46, 'Willamette Valley'), (47, 'Rogue Valley'), (49, 'Willamette Valley'), (50, 'Willamette Valley'), (51, 'Willamette Valley'), (52

In [437]:
cleaning_df['AVA'] = None
cleaning_df['Region'] = None
cleaning_df['Province'] = None
cleaning_df['Country'] = None

for i, entry in ava:
    cleaning_df.loc[i,"AVA"] = entry
    
for i, entry in region:
    cleaning_df.loc[i,'Region'] = entry
    
for i, entry in province:
    cleaning_df.loc[i,'Province'] = entry
    
for i, entry in country:
    cleaning_df.loc[i, 'Country'] = entry
    
    

In [438]:
cleaning_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741 entries, 0 to 740
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   price            739 non-null    object
 1   rating           741 non-null    int64 
 2   bottle_name      741 non-null    object
 3   region           741 non-null    object
 4   review_link      741 non-null    object
 5   source           741 non-null    object
 6   Designation      427 non-null    object
 7   Variety          741 non-null    object
 8   Appellation      741 non-null    object
 9   Winery           741 non-null    object
 10  Alcohol          531 non-null    object
 11  Bottle Size      741 non-null    object
 12  Category         741 non-null    object
 13  Date Published   741 non-null    object
 14  User Avg Rating  741 non-null    object
 15  vintage          730 non-null    object
 16  AVA              541 non-null    object
 17  Region           731 non-null    ob

In [234]:
## Some of the designations are grape varietals and regions, we can remove those for they already have their own column
for i in designation.index:
    if designation[i] in cleaning_df['Variety'].unique():
        designation[i] = None
    else:
        print(designation[i])
        

Rascal
Oregon Wine Co.
Laurene
Momtazi Vineyard
Unabridged
Ridgecrest Vineyard
Holstein Vineyard Blanc de Noir
Belle Pente Vineyard
Momtazi Vineyard
Sparkling Blanc de Blancs
Terroir Series
Isla
Cristo Irresisto Oregon Pink Wine
Zéphirine
Estate
RPG Vineyard
Brut Cuvée Méthode Champenoise
Crater View Vineyard & Seven Hills Vineyard Grenache-Syrah-Mourvèdre
Flood Line
Estate Blanc de Noir
Evolution
The Eyrie South Block
Estate
Extended Tirage Brut
El Torero
Redford-Wetle Vineyard Renardière
Rosé of
Celestina Vineyard
Fern Creek Vineyard
Delaney Vineyard
Front Yard Vineyard
Vintage Brut
Hyland Vineyard
Cattrall Vineyard
Willow Creek Vineyard
Khayam's Block
AVA
Nisse+Hygge Red Table Wine
RMS Brut
Coury Old Vine Single Clone
Sorella 60th
Il Rinnegati
La Colina Vineyard
Oregon Reserve
Old Vines
Belle Pente Vineyard
Coles Valley Vineyard Blanc de Noir
Coles Valley Vineyard
La Chouette Vineyard
Chehalem Mountain Vineyard
Funky Jory
Natasha Skov
Griffin Martin
Estate Reserve Select Cluster
Old

In [439]:
cleaning_df['Designation']  = designation
cleaning_df['Designation']

0                Rascal
1                   NaN
2       Oregon Wine Co.
3               Laurene
4      Momtazi Vineyard
             ...       
736                 NaN
737                 NaN
738     Clos Electrique
739                 NaN
740                 NaN
Name: Designation, Length: 741, dtype: object

In [450]:
cleaning_df['title'] = cleaning_df['bottle_name'].copy()

In [440]:
cleaning_df.head(10)

,price,rating,bottle_name,region,review_link,source,Designation,Variety,Appellation,Winery,Alcohol,Bottle Size,Category,Date Published,User Avg Rating,vintage,AVA,Region,Province,Country
0,$13,88,The Great Oregon Wine Company 2021 Rascal Pino...,Oregon Other,https://www.winemag.com/buying-guide/the-great...,Wine Enthusiast,Rascal,Pinot Noir,"['Oregon', ' Oregon Other', ' Oregon', ' US']",The Great Oregon Wine Company,13.3%,750 ml,['Red'],['10/1/2022'],['Not rated yet [Add Your Review]'],2021,None,Oregon Other,Oregon,US
1,$19,86,Wine by Joe 2019 Pinot Noir (Oregon),Oregon Other,https://www.winemag.com/buying-guide/wine-by-j...,Wine Enthusiast,NaN,Pinot Noir,"['Oregon', ' Oregon Other', ' Oregon', ' US']",Wine by Joe,13.5%,750 ml,['Red'],['10/1/2022'],['Not rated yet [Add Your Review]'],2019,None,Oregon Other,Oregon,US
2,$15,88,Duck Pond Cellars 2021 Oregon Wine Co. Chardon...,Oregon Other,https://www.winemag.com/buying-guide/duck-pond...,Wine Enthusiast,Oregon Wine Co.,Chardonnay,"['Oregon', ' Oregon Other', ' Oregon', ' US']",Duck Pond Cellars,13.2%,750 ml,['White'],['9/1/2022'],['Not rated yet [Add Your Review]'],2021,None,Oregon Other,Oregon,US
3,$75,92,Domaine Drouhin 2019 Laurene Pinot Noir (Dunde...,Willamette Valley,https://www.winemag.com/buying-guide/domaine-d...,Wine Enthusiast,Laurene,Pinot Noir,"['Dundee Hills', ' Willamette Valley', ' Orego...",Domaine Drouhin,13.9%,750 ml,['Red'],['7/1/2022'],['Not rated yet [Add Your Review]'],2019,Dundee Hills,Willamette Valley,Oregon,US
4,$40,90,Portlandia 2019 Momtazi Vineyard Pinot Noir (W...,Willamette Valley,https://www.winemag.com/buying-guide/portlandi...,Wine Enthusiast,Momtazi Vineyard,Pinot Noir,"['Willamette Valley', ' Willamette Valley', ' ...",Portlandia,13.5%,750 ml,['Red'],['7/1/2022'],['Not rated yet [Add Your Review]'],2019,Willamette Valley,Willamette Valley,Oregon,US
5,$33,88,Matzinger Davies 2018 Unabridged Sauvignon Bla...,Oregon Other,https://www.winemag.com/buying-guide/matzinger...,Wine Enthusiast,Unabridged,Sauvignon Blanc,"['Columbia Gorge (OR)', ' Oregon Other', ' Ore...",Matzinger Davies,13.5%,750 ml,['White'],['7/1/2022'],['Not rated yet [Add Your Review]'],2018,Columbia Gorge (OR),Oregon Other,Oregon,US
6,$33,93,Élevée Winegrowers 2020 Ridgecrest Vineyard Ri...,Willamette Valley,https://www.winemag.com/buying-guide/elevee-wi...,Wine Enthusiast,Ridgecrest Vineyard,Riesling,"['Ribbon Ridge', ' Willamette Valley', ' Orego...",Élevée Winegrowers,11.5%,750 ml,['White'],['5/1/2022'],['Not rated yet [Add Your Review]'],2020,Ribbon Ridge,Willamette Valley,Oregon,US
7,$90,91,Granville 2017 Holstein Vineyard Blanc de Noir...,Willamette Valley,https://www.winemag.com/buying-guide/granville...,Wine Enthusiast,Holstein Vineyard Blanc de Noir,Sparkling Blend,"['Dundee Hills', ' Willamette Valley', ' Orego...",Granville,12%,750 ml,['Sparkling'],['5/1/2022'],['Not rated yet [Add Your Review]'],2017,Dundee Hills,Willamette Valley,Oregon,US
8,$25,88,Ashlyn 2019 Pinot Noir (Willamette Valley),Willamette Valley,https://www.winemag.com/buying-guide/ashlyn-20...,Wine Enthusiast,NaN,Pinot Noir,"['Willamette Valley', ' Willamette Valley', ' ...",Ashlyn,13.9%,750 ml,['Red'],['5/1/2022'],['Not rated yet [Add Your Review]'],2019,Willamette Valley,Willamette Valley,Oregon,US
9,$55,97,Domaine Divio 2020 Chardonnay (Willamette Valley),Willamette Valley,https://www.winemag.com/buying-guide/domaine-d...,Wine Enthusiast,NaN,Chardonnay,"['Willamette Valley', ' Willamette Valley', ' ...",Domaine Divio,13.3%,750 ml,['White'],['4/1/2022'],['Not rated yet [Add Your Review]'],2020,Willamette Valley,Willamette Valley,Oregon,US


In [441]:
category = cleaning_df['Category'].copy()
# cleaning_df['Category'] = enthusiast_df['Category'].copy()
for i in category.index:
    category[i] = category[i][2:-2]

category

0        Red
1        Red
2      White
3        Red
4        Red
       ...  
736    White
737    White
738    White
739    White
740    White
Name: Category, Length: 741, dtype: object

In [442]:
cleaning_df['Category'] = category

In [443]:
cleaning_df['User Avg Rating'].value_counts() ## Dropping that column

['Not rated yet [Add Your Review]']    741
Name: User Avg Rating, dtype: int64

In [444]:
review_date = cleaning_df["Date Published"].copy()
for i in review_date.index:
    review_date[i] = review_date[i][2:-2]
review_date

0       10/1/2022
1       10/1/2022
2        9/1/2022
3        7/1/2022
4        7/1/2022
          ...    
736      4/1/2000
737    12/31/1999
738     11/1/1999
739     11/1/1999
740      8/1/1999
Name: Date Published, Length: 741, dtype: object

In [445]:
cleaning_df['Date Published'] = review_date
cleaning_df['Date Published']

0       10/1/2022
1       10/1/2022
2        9/1/2022
3        7/1/2022
4        7/1/2022
          ...    
736      4/1/2000
737    12/31/1999
738     11/1/1999
739     11/1/1999
740      8/1/1999
Name: Date Published, Length: 741, dtype: object

In [446]:
cleaning_df['Varietal'] = cleaning_df['Variety'].copy()
cleaning_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741 entries, 0 to 740
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   price            739 non-null    object
 1   rating           741 non-null    int64 
 2   bottle_name      741 non-null    object
 3   region           741 non-null    object
 4   review_link      741 non-null    object
 5   source           741 non-null    object
 6   Designation      427 non-null    object
 7   Variety          741 non-null    object
 8   Appellation      741 non-null    object
 9   Winery           741 non-null    object
 10  Alcohol          531 non-null    object
 11  Bottle Size      741 non-null    object
 12  Category         741 non-null    object
 13  Date Published   741 non-null    object
 14  User Avg Rating  741 non-null    object
 15  vintage          730 non-null    object
 16  AVA              541 non-null    object
 17  Region           731 non-null    ob

In [452]:
clean_enthusiast_df = cleaning_df.loc[:,['price', 'rating', 'title', 'source','Designation', 'Winery', 
'Category', 'vintage', 'AVA', 'Region', 'Province', 'Country', 'Varietal']]
clean_enthusiast_df.columns = [x.lower() for x in clean_enthusiast_df.columns]
clean_enthusiast_df

,price,rating,title,source,designation,winery,category,vintage,ava,region,province,country,varietal
0,$13,88,The Great Oregon Wine Company 2021 Rascal Pino...,Wine Enthusiast,Rascal,The Great Oregon Wine Company,Red,2021,None,Oregon Other,Oregon,US,Pinot Noir
1,$19,86,Wine by Joe 2019 Pinot Noir (Oregon),Wine Enthusiast,NaN,Wine by Joe,Red,2019,None,Oregon Other,Oregon,US,Pinot Noir
2,$15,88,Duck Pond Cellars 2021 Oregon Wine Co. Chardon...,Wine Enthusiast,Oregon Wine Co.,Duck Pond Cellars,White,2021,None,Oregon Other,Oregon,US,Chardonnay
3,$75,92,Domaine Drouhin 2019 Laurene Pinot Noir (Dunde...,Wine Enthusiast,Laurene,Domaine Drouhin,Red,2019,Dundee Hills,Willamette Valley,Oregon,US,Pinot Noir
4,$40,90,Portlandia 2019 Momtazi Vineyard Pinot Noir (W...,Wine Enthusiast,Momtazi Vineyard,Portlandia,Red,2019,Willamette Valley,Willamette Valley,Oregon,US,Pinot Noir
...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,$18,90,Ponzi 1998 Arneis (Willamette Valley),Wine Enthusiast,NaN,Ponzi,White,1998,Willamette Valley,Willamette Valley,Oregon,US,Arneis
737,$12,86,Tyee 1996 Pinot Blanc (Willamette Valley),Wine Enthusiast,NaN,Tyee,White,1996,Willamette Valley,Willamette Valley,Oregon,US,Pinot Blanc
738,$35,92,Cameron 1997 Clos Electrique Chardonnay (Willa...,Wine Enthusiast,Clos Electrique,Cameron,White,1997,Willamette Valley,Willamette Valley,Oregon,US,Chardonnay
739,$15,90,Torii Mor 1998 Pinot Gris (Yamhill County),Wine Enthusiast,NaN,Torii Mor,White,1998,Yamhill County,Willamette Valley,Oregon,US,Pinot Gris


In [453]:
clean_enthusiast_df.to_csv("cleaned_wine_enthusiast_reviews.csv", index=False)

## Add the Wine Enthusiast reviews to the sql Database

In [454]:
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.automap import automap_base
from config import *

db_address = db_type + db_user + db_pass + db_project + db_aws

In [456]:
engine = create_engine(db_address)

In [457]:
clean_enthusiast_df.to_sql(name='wine_reviews', con=engine, if_exists='replace', index=True, index_label="review_id", chunksize=50)

In [458]:
conn = engine.connect()

In [459]:
[x for x in conn.execute("Select count(*) from wine_reviews;")]

[(5373,)]

In [460]:
conn.close()

## Cleaning Update

Wine Enthusiast reviews are clean enough to merge. I see that there is no review text saved for these but there are on the webpage. This will have to be remedied at a later date.


- When all the reviews are combined into one table the following columns are needed for filtering the data and creating the list of unique wines:  Vintage, Category, Varietal, State, Region, AVA, Winery, bottle_name, designation


In [347]:
spectator_df.columns

Index(['vintage', 'score', 'release_price', 'bottle_name', 'winery',
       'review_text', 'Magazine', 'review_link', 'region', 'source',
       'varietal', 'wine_id', 'Region'],
      dtype='object')

In [461]:
clean_enthusiast_df.columns

Index(['price', 'rating', 'title', 'source', 'designation', 'winery',
       'category', 'vintage', 'ava', 'region', 'province', 'country',
       'varietal'],
      dtype='object')

## Notes for combining the 2 dataframes    

Wine Enthusiast
- confirm bottle_name is broken into the following columns: Vintage, Winery, Varietal, Region/AVA, Designation then do not include it in the unique wine list. 

Wine Spectator
- add magazine issue into the source column


Combined Table
- create: unique review id, unique wine id

Extracted unique wine list
- columns: Winery, Vintage, designation, Varietal, AVA, region, province, Country, Category

- create: unique wine id column

In [360]:
for i in clean_enthusiast_df.index:
    print(clean_enthusiast_df["Winery"][i], "   ||   ", clean_enthusiast_df["vintage"][i], "   ||   ", clean_enthusiast_df["Designation"][i], "   ||   ",  clean_enthusiast_df["Varietal"][i], "   ||   " )
    print(clean_enthusiast_df['title'][i])
    print("===========")

The Great Oregon Wine Company    ||    2021    ||    Rascal    ||    Pinot Noir    ||   
The Great Oregon Wine Company 2021 Rascal Pinot Noir (Oregon)
Wine by Joe    ||    2019    ||    nan    ||    Pinot Noir    ||   
Wine by Joe 2019 Pinot Noir (Oregon)
Duck Pond Cellars    ||    2021    ||    Oregon Wine Co.    ||    Chardonnay    ||   
Duck Pond Cellars 2021 Oregon Wine Co. Chardonnay (Oregon)
Domaine Drouhin    ||    2019    ||    Laurene    ||    Pinot Noir    ||   
Domaine Drouhin 2019 Laurene Pinot Noir (Dundee Hills)
Portlandia    ||    2019    ||    Momtazi Vineyard    ||    Pinot Noir    ||   
Portlandia 2019 Momtazi Vineyard Pinot Noir (Willamette Valley)
Matzinger Davies    ||    2018    ||    Unabridged    ||    Sauvignon Blanc    ||   
Matzinger Davies 2018 Unabridged Sauvignon Blanc (Columbia Gorge (OR))
Élevée Winegrowers    ||    2020    ||    Ridgecrest Vineyard    ||    Riesling    ||   
Élevée Winegrowers 2020 Ridgecrest Vineyard Riesling (Ribbon Ridge)
Granville  

Lemelson    ||    2016    ||    Reserve    ||    Chardonnay    ||   
Lemelson 2016 Reserve Chardonnay (Willamette Valley)
Foris    ||    2018    ||    nan    ||    Pinot Gris    ||   
Foris 2018 Pinot Gris (Rogue Valley)
Martin Woods    ||    2017    ||    Jessie James Vineyard    ||    Pinot Noir    ||   
Martin Woods 2017 Jessie James Vineyard Pinot Noir (McMinnville)
Foris    ||    2016    ||    nan    ||    Tempranillo    ||   
Foris 2016 Tempranillo (Rogue Valley)
Anne Amie    ||    2017    ||    Twelve Oaks Estate    ||     Gamay    ||   
Anne Amie 2017 Twelve Oaks Estate Gamay Noir (Chehalem Mountains)
Maison L'Envoyé    ||    2018    ||    Straight Shooter    ||    Chardonnay    ||   
Maison L'Envoyé 2018 Straight Shooter Chardonnay (Willamette Valley)
Capitello    ||    2018    ||    Croft Vineyard    ||    Sauvignon Blanc    ||   
Capitello 2018 Croft Vineyard Sauvignon Blanc (Willamette Valley)
Anchor Valley    ||    2017    ||    nan    ||    Pinot Noir    ||   
Anchor Vall

Armonéa    ||    2014    ||    Summit Vineyard Estate    ||    Pinot Noir    ||   
Armonéa 2014 Summit Vineyard Estate Pinot Noir (Dundee Hills)
Apolloni    ||    2016    ||    Estate    ||    Pinot Blanc    ||   
Apolloni 2016 Estate Pinot Blanc (Willamette Valley)
Alloro    ||    2016    ||    nan    ||    Riesling    ||   
Alloro 2016 Riesling (Chehalem Mountains)
Abacela    ||    2014    ||    Reserve South Face Block    ||    Syrah    ||   
Abacela 2014 Reserve South Face Block Syrah (Umpqua Valley)
Helioterra    ||    2016    ||    Redman Vineyard    ||    Arneis    ||   
Helioterra 2016 Redman Vineyard Arneis (Ribbon Ridge)
Argyle    ||    2014    ||    Grower Series Vintage Brut    ||    Blend    ||   
Argyle 2014 Grower Series Vintage Brut Sparkling (Willamette Valley)
Iris Vineyards    ||    2015    ||    Chalice Estate    ||    Chardonnay    ||   
Iris Vineyards 2015 Chalice Estate Chardonnay (Willamette Valley)
Duck Pond    ||    2016    ||    nan    ||     Gamay    ||   
D

Ardiri    ||    2012    ||    nan    ||    Pinot Noir    ||   
Ardiri 2012 Pinot Noir (California)
Sabina Vineyards    ||    2011    ||    nan    ||    Pinot Noir    ||   
Sabina Vineyards 2011 Pinot Noir (Willamette Valley)
Harper Voit    ||    2013    ||    Bieze Vineyard    ||    Pinot Noir    ||   
Harper Voit 2013 Bieze Vineyard Pinot Noir (Eola-Amity Hills)
Quady North    ||    2011    ||    Steelhead Run Vineyard    ||    Syrah    ||   
Quady North 2011 Steelhead Run Vineyard Syrah (Applegate Valley)
Treos    ||    2014    ||    nan    ||    Albariño    ||   
Treos 2014 Albariño (Willamette Valley)
J. Wright    ||    2012    ||    nan    ||    Pinot Noir    ||   
J. Wright 2012 Pinot Noir (Willamette Valley)
Animale    ||    2012    ||    nan    ||    Pinot Noir    ||   
Animale 2012 Pinot Noir (Oregon)
J. Wright    ||    2011    ||    nan    ||    Pinot Noir    ||   
J. Wright 2011 Pinot Noir (Willamette Valley)
Delfino    ||    2012    ||    nan    ||    Zinfandel    ||   
Del

William Hatcher    ||    2007    ||    nan    ||    Pinot Noir    ||   
William Hatcher 2007 Pinot Noir (Willamette Valley)
Lumos    ||    2009    ||    Rudolfo Vineyard    ||    Pinot Gris    ||   
Lumos 2009 Rudolfo Vineyard Pinot Gris (Willamette Valley)
RouteStock    ||    2009    ||    99W    ||    Pinot Noir    ||   
RouteStock 2009 99W Pinot Noir (Willamette Valley)
Macindoe    ||    2009    ||    None    ||    Melon    ||   
Macindoe 2009 Melon de Bourgogne Melon (Yamhill-Carlton)
Castle Rock    ||    2009    ||    nan    ||    Pinot Noir    ||   
Castle Rock 2009 Pinot Noir (Willamette Valley)
Dowsett Family    ||    2009    ||    Celilo Vineyard    ||    Gewürztraminer    ||   
Dowsett Family 2009 Celilo Vineyard Gewürztraminer (Columbia Gorge (WA))
Adelsheim    ||    2008    ||    Bryan Creek Vineyard    ||    Pinot Noir    ||   
Adelsheim 2008 Bryan Creek Vineyard Pinot Noir (Chehalem Mountains)
Cowhorn    ||    2008    ||    Syrah 74    ||    Syrah    ||   
Cowhorn 2008 Sy

Amity    ||    2000    ||    Eco-Wine    ||    Pinot Noir    ||   
Amity 2000 Eco-Wine Pinot Noir (Oregon)
Abacela    ||    2000    ||    nan    ||    Malbec    ||   
Abacela 2000 Malbec (Umpqua Valley)
Patton Valley    ||    2000    ||    nan    ||    Pinot Noir    ||   
Patton Valley 2000 Pinot Noir (Oregon)
Amity    ||    2001    ||    Eco-Wine    ||    Pinot Noir    ||   
Amity 2001 Eco-Wine Pinot Noir (Oregon)
Griffin Creek    ||    2001    ||    nan    ||    Pinot Gris    ||   
Griffin Creek 2001 Pinot Gris (Rogue Valley)
Willamette Valley Vineyards    ||    2001    ||    Founders' Reserve    ||    Pinot Gris    ||   
Willamette Valley Vineyards 2001 Founders' Reserve Pinot Gris (Willamette Valley)
Sebastiani    ||    1999    ||    nan    ||    Pinot Noir    ||   
Sebastiani 1999 Pinot Noir (Russian River Valley)
Archery Summit    ||    1999    ||    Red Hills Estate    ||    Pinot Noir    ||   
Archery Summit 1999 Red Hills Estate Pinot Noir (Willamette Valley)
Cambria    ||    

## Start cleaning Wine Spectator reviews

In [675]:
cleaning_df = spectator_df.copy()
cleaning_df

,vintage,score,release_price,bottle_name,winery,review_text,Magazine,review_link,region,source,varietal,wine_id,Region
0,2011,91,$40,Cabernet Franc Rogue Valley,Griffin Creek,"Soft, ripe, generous and precise, this open-te...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Franc,379875.0,NaN
1,2011,86,$40,Arsenal Rogue Valley,Quady North,"A snappy red, if modest in scale, with blackbe...","Magazine Issue Feb 02, 2017",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Franc,429948.0,NaN
2,2011,90,$28,Cabernet Sauvignon Rogue Valley,Del Rio,"A vibrant, medium-weight red, with ripe plum a...","Magazine Issue Feb 28, 2014",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Sauvignon & Blends,357733.0,NaN
3,2011,86,$40,Cabernet Sauvignon Rogue Valley,Griffin Creek,"Lean, spicy and firmly tannic, offering billow...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Sauvignon & Blends,379943.0,NaN
4,2011,94,$125,Chardonnay Dundee Hills Récolte Grand Cru,Domaine Serene,"Tangy, with vibrant acidity and expressive, ri...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Dundee Hills']",Wine Spectator,Chardonnay,380170.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,2015,90,$45,Chardonnay Dundee Hills,Knudsen,"Crisp and harmonious, with lemon blossom and a...",Magazine Issue Web Only - 2017,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Dundee Hills']",Wine Spectator,Chardonnay,434673.0,NaN
1234,2015,90,$40,Chardonnay Willamette Valley Reserve,Ponzi,"Graceful and polished, with expressive pear, s...","Magazine Issue Jul 31, 2018",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator,Chardonnay,454785.0,NaN
1235,2015,90,$45,Chardonnay Yamhill-Carlton District Gran Morai...,Raptor Ridge,"Light and polished, showing pretty pear, pinea...",Magazine Issue Web Only - 2017,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt...",Wine Spectator,Chardonnay,427274.0,NaN
1236,2015,90,$60,Chardonnay Yamhill-Carlton District Marsh Esta...,Roco,"Floral and harmonious, with delicate tropical ...",Magazine Issue Web Only - 2018,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt...",Wine Spectator,Chardonnay,447448.0,NaN


In [676]:
cleaning_df["source"] = cleaning_df['source'].copy() +" " +cleaning_df['Magazine'].copy()
cleaning_df['source']

0          Wine Spectator Magazine Issue Feb 28, 2015
1          Wine Spectator Magazine Issue Feb 02, 2017
2          Wine Spectator Magazine Issue Feb 28, 2014
3          Wine Spectator Magazine Issue Feb 28, 2015
4          Wine Spectator Magazine Issue Feb 28, 2015
                            ...                      
1233    Wine Spectator Magazine Issue Web Only - 2017
1234       Wine Spectator Magazine Issue Jul 31, 2018
1235    Wine Spectator Magazine Issue Web Only - 2017
1236    Wine Spectator Magazine Issue Web Only - 2018
1237    Wine Spectator Magazine Issue Web Only - 2020
Name: source, Length: 1238, dtype: object

In [677]:
cleaning_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1238 entries, 0 to 1237
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   vintage        1238 non-null   int64  
 1   score          1238 non-null   int64  
 2   release_price  1238 non-null   object 
 3   bottle_name    1238 non-null   object 
 4   winery         1238 non-null   object 
 5   review_text    1238 non-null   object 
 6   Magazine       1238 non-null   object 
 7   review_link    1238 non-null   object 
 8   region         865 non-null    object 
 9   source         1238 non-null   object 
 10  varietal       865 non-null    object 
 11  wine_id        865 non-null    float64
 12  Region         373 non-null    object 
dtypes: float64(1), int64(2), object(10)
memory usage: 125.9+ KB


In [678]:
regions = cleaning_df['region'].dropna().copy()
regions

0           ['Oregon', 'Southern Oregon', 'Rogue Valley']
1           ['Oregon', 'Southern Oregon', 'Rogue Valley']
2           ['Oregon', 'Southern Oregon', 'Rogue Valley']
3           ['Oregon', 'Southern Oregon', 'Rogue Valley']
4         ['Oregon', 'Willamette Valley', 'Dundee Hills']
                              ...                        
1233      ['Oregon', 'Willamette Valley', 'Dundee Hills']
1234                      ['Oregon', 'Willamette Valley']
1235    ['Oregon', 'Willamette Valley', 'Yamhill-Carlt...
1236    ['Oregon', 'Willamette Valley', 'Yamhill-Carlt...
1237                      ['Oregon', 'Willamette Valley']
Name: region, Length: 865, dtype: object

In [679]:
province = []
ava = []
region = []

for i in regions.index:
    bottle = regions[i][2:-2].split("', '")
    province.append((i, bottle[0]))
    if len(bottle) != 3:
        if len(bottle) == 2:
            region.append((i, bottle[1]))
    else:
        region.append((i, bottle[1]))
        ava.append((i, bottle[-1]))
print(province, "\n=====================\n")    
print(region, "\n=====================\n")    
print(ava, "\n=====================\n")    

[(0, 'Oregon'), (1, 'Oregon'), (2, 'Oregon'), (3, 'Oregon'), (4, 'Oregon'), (5, 'Oregon'), (6, 'Oregon'), (7, 'Oregon'), (8, 'Oregon'), (9, 'Oregon'), (10, 'Oregon'), (11, 'Oregon'), (12, 'Oregon'), (13, 'Oregon'), (14, 'Oregon'), (15, 'Oregon'), (16, 'Oregon'), (17, 'Oregon'), (18, 'Oregon'), (19, 'Oregon'), (20, 'Oregon'), (21, 'Oregon'), (22, 'Oregon'), (23, 'Oregon'), (24, 'Oregon'), (25, 'Oregon'), (26, 'Oregon'), (27, 'Oregon'), (28, 'Oregon'), (29, 'Oregon'), (30, 'Oregon'), (31, 'Oregon'), (32, 'Oregon'), (33, 'Oregon'), (34, 'Oregon'), (35, 'Oregon'), (36, 'Oregon'), (37, 'Oregon'), (38, 'Oregon'), (39, 'Oregon'), (40, 'Oregon'), (41, 'Oregon'), (42, 'Oregon'), (43, 'Oregon'), (44, 'Oregon'), (45, 'Oregon'), (46, 'Oregon'), (47, 'Oregon'), (48, 'Oregon'), (49, 'Oregon'), (50, 'Oregon'), (51, 'Oregon'), (52, 'Oregon'), (53, 'Oregon'), (54, 'Oregon'), (55, 'Oregon'), (56, 'Oregon'), (57, 'Oregon'), (58, 'Oregon'), (59, 'Oregon'), (60, 'Oregon'), (61, 'Oregon'), (62, 'Oregon'), (

In [680]:
cleaning_df["AVA"] = None
cleaning_df['Region'] = None
cleaning_df['Province'] = None

for i, entry in ava:
    cleaning_df.loc[i, 'AVA'] = entry
    
for i, entry in region:
    cleaning_df.loc[i, 'Region'] = entry
    
for i, entry in province:
    cleaning_df.loc[i, 'Province'] = entry

cleaning_df.loc[:,["region", "AVA", "Region", "Province"]]

,region,AVA,Region,Province
0,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Rogue Valley,Southern Oregon,Oregon
1,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Rogue Valley,Southern Oregon,Oregon
2,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Rogue Valley,Southern Oregon,Oregon
3,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Rogue Valley,Southern Oregon,Oregon
4,"['Oregon', 'Willamette Valley', 'Dundee Hills']",Dundee Hills,Willamette Valley,Oregon
...,...,...,...,...
1233,"['Oregon', 'Willamette Valley', 'Dundee Hills']",Dundee Hills,Willamette Valley,Oregon
1234,"['Oregon', 'Willamette Valley']",None,Willamette Valley,Oregon
1235,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt...",Yamhill-Carlton District,Willamette Valley,Oregon
1236,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt...",Yamhill-Carlton District,Willamette Valley,Oregon


In [683]:
for i in cleaning_df.index:
    print(cleaning_df['bottle_name'][i], "  ||  ", cleaning_df['varietal'][i])

Cabernet Franc Rogue Valley   ||   Cabernet Franc
Arsenal Rogue Valley   ||   Cabernet Franc
Cabernet Sauvignon Rogue Valley   ||   Cabernet Sauvignon & Blends
Cabernet Sauvignon Rogue Valley   ||   Cabernet Sauvignon & Blends
Chardonnay Dundee Hills Récolte Grand Cru   ||   Chardonnay
Chardonnay Eola-Amity Hills Seven Springs Vineyard La Source   ||   Chardonnay
Chardonnay Willamette Valley Sigrid   ||   Chardonnay
Chardonnay Dundee Hills Édition Limitée   ||   Chardonnay
Chardonnay Dundee Hills Evenstad Reserve   ||   Chardonnay
Chardonnay Dundee Hills   ||   Chardonnay
Chardonnay Eola-Amity Hills Seven Springs Vineyard   ||   Chardonnay
Chardonnay Eola-Amity Hills Nuthouse Reserve Series   ||   Chardonnay
Chardonnay Willamette Valley Old Stones   ||   Chardonnay
Chardonnay Willamette Valley   ||   Chardonnay
Chardonnay Ribbon Ridge Cascadia   ||   Chardonnay
Chardonnay Dundee Hills Stoller Vineyards Ian's Reserve   ||   Chardonnay
Chardonnay Dundee Hills Côte Sud Vineyard Dijon Clon

Pinot Noir Eola-Amity Hills Seven Springs Vineyard La Source   ||   nan
Pinot Noir Ribbon Ridge The Upper Terrace   ||   nan
Chardonnay Eola-Amity Hills Seven Springs Vineyard La Source   ||   nan
Chardonnay Eola-Amity Hills Seven Springs Vineyard Summum   ||   nan
Pinot Noir Eola-Amity Hills Seven Springs Vineyard   ||   nan
Pinot Noir Chehalem Mountains Quarter Mile Lane Vineyard   ||   nan
Pinot Noir Dundee Hills Signature   ||   nan
Pinot Noir Ribbon Ridge The Beaux Frères Vineyard   ||   nan
Pinot Noir Chehalem Mountains Winery Block   ||   nan
Pinot Noir Dundee Hills Bergström Vineyard   ||   nan
Pinot Noir Eola-Amity Hills Flat Block   ||   nan
Chardonnay Willamette Valley   ||   nan
Pinot Noir Willamette Valley   ||   nan
Pinot Noir Willamette Valley   ||   nan
Pinot Noir Ribbon Ridge Evelyn's   ||   nan
Pinot Noir Yamhill-Carlton District Mineral Springs Ranch   ||   nan
Pinot Noir Dundee Hills Helen's   ||   nan
Pinot Noir Chehalem Mountains Boulder Bluff Vineyard   ||   nan


Chardonnay Willamette Valley Old Stones   ||   Chardonnay
Chardonnay Eola-Amity Hills Casteel   ||   Chardonnay
Chardonnay Ribbon Ridge Cascadia   ||   Chardonnay
Chardonnay Chehalem Mountains   ||   Chardonnay
Chardonnay Dundee Hills Arthur   ||   Chardonnay
Chardonnay Dundee Hills Récolte Grand Cru   ||   Chardonnay
Chardonnay Yamhill-Carlton Goodrich   ||   Chardonnay
Chardonnay Chehalem Mountains Chehalem Mountain Vineyard   ||   Chardonnay
Chardonnay Yamhill-Carlton   ||   Chardonnay
Chardonnay Willamette Valley Avni   ||   Chardonnay
Chardonnay Willamette Valley   ||   Chardonnay
Chardonnay Dundee Hills Latchkey Vineyard   ||   Chardonnay
Chardonnay McMinnville Hyland Vineyard   ||   Chardonnay
Chardonnay Dundee Hills Maresh   ||   Chardonnay
Chardonnay Willamette Valley The Wild Bee   ||   Chardonnay
Chardonnay Dundee Hills Evenstad Reserve   ||   Chardonnay
Chardonnay Eola-Amity Hills   ||   Chardonnay
Chardonnay Eola-Amity Hills Eola Springs Vineyard   ||   Chardonnay
Chardonn

In [684]:
varietal = list(cleaning_df['varietal'].unique())
varietal

['Cabernet Franc',
 'Cabernet Sauvignon & Blends',
 'Chardonnay',
 'Dessert Wines',
 'Grenache',
 'Malbec',
 'Other Red Wines',
 'Other White Wines',
 'Pinot Blanc',
 'Pinot Gris',
 'Riesling',
 'Rosé Wines',
 'Sauvignon Blanc',
 'Shiraz/Syrah',
 'Sparkling Wines',
 'Tempranillo',
 'Viognier',
 'Pinot Noir',
 'Grüner Veltliner',
 nan,
 'Gewürztraminer',
 'Merlot']

In [685]:
varietal.pop(-3)
varietal.pop(3)
varietal.pop(5)
varietal.pop(5)
varietal.pop(8)
varietal.pop(10)
varietal

['Cabernet Franc',
 'Cabernet Sauvignon & Blends',
 'Chardonnay',
 'Grenache',
 'Malbec',
 'Pinot Blanc',
 'Pinot Gris',
 'Riesling',
 'Sauvignon Blanc',
 'Shiraz/Syrah',
 'Tempranillo',
 'Viognier',
 'Pinot Noir',
 'Grüner Veltliner',
 'Gewürztraminer',
 'Merlot']

In [686]:
varietal[1] = varietal[1].split(" & ")[0]
varietal.append(varietal[9].split('/')[0])
varietal[9] = varietal[9].split('/')[1]
varietal

['Cabernet Franc',
 'Cabernet Sauvignon',
 'Chardonnay',
 'Grenache',
 'Malbec',
 'Pinot Blanc',
 'Pinot Gris',
 'Riesling',
 'Sauvignon Blanc',
 'Syrah',
 'Tempranillo',
 'Viognier',
 'Pinot Noir',
 'Grüner Veltliner',
 'Gewürztraminer',
 'Merlot',
 'Shiraz']

In [687]:
ava = list(cleaning_df['AVA'].unique())
region = list(cleaning_df['Region'].unique())
ava[3] = region[0]
ava[-2] = region[1]
ava

['Rogue Valley',
 'Dundee Hills',
 'Eola-Amity Hills',
 'Southern Oregon',
 'Ribbon Ridge',
 'Chehalem Mountains',
 'Applegate Valley',
 'Yamhill-Carlton District',
 'Umpqua Valley',
 'McMinnville',
 'Willamette Valley',
 'Van Duzer Corridor']

In [689]:
cleaning_df['category'] = None
cleaning_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1238 entries, 0 to 1237
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   vintage        1238 non-null   int64  
 1   score          1238 non-null   int64  
 2   release_price  1238 non-null   object 
 3   bottle_name    1238 non-null   object 
 4   winery         1238 non-null   object 
 5   review_text    1238 non-null   object 
 6   Magazine       1238 non-null   object 
 7   review_link    1238 non-null   object 
 8   region         865 non-null    object 
 9   source         1238 non-null   object 
 10  varietal       865 non-null    object 
 11  wine_id        865 non-null    float64
 12  Region         778 non-null    object 
 13  AVA            481 non-null    object 
 14  Province       865 non-null    object 
 15  category       0 non-null      object 
dtypes: float64(1), int64(2), object(13)
memory usage: 154.9+ KB


In [690]:
cleaning_df.columns

Index(['vintage', 'score', 'release_price', 'bottle_name', 'winery',
       'review_text', 'Magazine', 'review_link', 'region', 'source',
       'varietal', 'wine_id', 'Region', 'AVA', 'Province', 'category'],
      dtype='object')

In [692]:
## Title Format is Grape(s), AVA/Region, Designation

for i in cleaning_df.index:
    title = cleaning_df.loc[i,'bottle_name']
    for a in ava:
        if a in title:
            unpack = title.split(" "+  a)
            grape = unpack[0]
            des = unpack[1:]   # designation
            cleaning_df.loc[i, 'AVA'] = a
            if cleaning_df.loc[i,'varietal'] == "Dessert Wines":
                cleaning_df.loc[i, 'category'] = "Dessert"
                cleaning_df.loc[i,'varietal'] = grape
                pass
            elif grape == 'Brut':
                cleaning_df.loc[i, 'category'] = "Sparkling"
                pass
            elif grape == "Rosé":
                cleaning_df.loc[i,'varietal'] = 'blend'
                cleaning_df.loc[i,'category'] = 'Rosé'
                pass 
            else:
                cleaning_df.loc[i,'varietal'] = grape
            
            if len(des) > 1:
                if (len(des[0]) == 0):
                    cleaning_df.loc[i,'designation'] = des[1][1:]
                else:
                    cleaning_df.loc[i,'variety'] = str(des)[3:-2].replace("', '", "")
            elif len(des) < 1:
                pass # des is an empty list
            else:
                des = des[0][1:]
                
                if des == '':
                    pass
                elif des == "Rosé":
                    cleaning_df.loc[i, 'designation'] = 'Rosé'
                    cleaning_df.loc[i, 'category'] = 'Rosé'
                else:
                    cleaning_df.loc[i,'designation'] = des
                    
                    
cleaning_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1238 entries, 0 to 1237
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   vintage        1238 non-null   int64  
 1   score          1238 non-null   int64  
 2   release_price  1238 non-null   object 
 3   bottle_name    1238 non-null   object 
 4   winery         1238 non-null   object 
 5   review_text    1238 non-null   object 
 6   Magazine       1238 non-null   object 
 7   review_link    1238 non-null   object 
 8   region         865 non-null    object 
 9   source         1238 non-null   object 
 10  varietal       1221 non-null   object 
 11  wine_id        865 non-null    float64
 12  Region         778 non-null    object 
 13  AVA            1131 non-null   object 
 14  Province       865 non-null    object 
 15  category       83 non-null     object 
 16  designation    805 non-null    object 
 17  variety        2 non-null      object 
dtypes: float

In [695]:
clean_enthusiast_df.columns

Index(['price', 'rating', 'title', 'source', 'designation', 'winery',
       'category', 'vintage', 'ava', 'region', 'province', 'country',
       'varietal'],
      dtype='object')

In [693]:
cleaning_df['title'] = None
cleaning_df['country'] = 'US'
cleaning_df['rating'] = cleaning_df['score'].copy()
cleaning_df['description'] = cleaning_df['review_text'].copy()
cleaning_df['variety'] = cleaning_df['varietal'].copy()
cleaning_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1238 entries, 0 to 1237
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   vintage        1238 non-null   int64  
 1   score          1238 non-null   int64  
 2   release_price  1238 non-null   object 
 3   bottle_name    1238 non-null   object 
 4   winery         1238 non-null   object 
 5   review_text    1238 non-null   object 
 6   Magazine       1238 non-null   object 
 7   review_link    1238 non-null   object 
 8   region         865 non-null    object 
 9   source         1238 non-null   object 
 10  varietal       1221 non-null   object 
 11  wine_id        865 non-null    float64
 12  Region         778 non-null    object 
 13  AVA            1131 non-null   object 
 14  Province       865 non-null    object 
 15  category       83 non-null     object 
 16  designation    805 non-null    object 
 17  variety        1221 non-null   object 
 18  title   

In [694]:
for i in cleaning_df.index:
    cleaning_df.loc[i, 'title'] = cleaning_df.loc[i,'winery']+ " " +str(cleaning_df.loc[i,'vintage'])+ " " + cleaning_df.loc[i,'bottle_name']

cleaning_df['title']

0          Griffin Creek 2011 Cabernet Franc Rogue Valley
1                   Quady North 2011 Arsenal Rogue Valley
2            Del Rio 2011 Cabernet Sauvignon Rogue Valley
3       Griffin Creek 2011 Cabernet Sauvignon Rogue Va...
4       Domaine Serene 2011 Chardonnay Dundee Hills Ré...
                              ...                        
1233                 Knudsen 2015 Chardonnay Dundee Hills
1234      Ponzi 2015 Chardonnay Willamette Valley Reserve
1235    Raptor Ridge 2015 Chardonnay Yamhill-Carlton D...
1236    Roco 2015 Chardonnay Yamhill-Carlton District ...
1237    Argyle 2015 Brut Rosé Willamette Valley Artisa...
Name: title, Length: 1238, dtype: object

In [699]:
clean_spectator_df = cleaning_df[['winery','vintage','variety', 'designation', 'release_price', 'rating', 'title', 'source', 
       'category',  'AVA', 'Region', 'Province', 'country', 'description']]
# # ['winery', 'vintage', 'designation',  'variety', 'rating', 'category','title',
# #               'AVA', 'region','province','country',  'release_price', 'description', 'source']
clean_spectator_df.columns = [x.lower() if x != 'AVA' else x for x in list(clean_spectator_df.columns)]
clean_spectator_df

,winery,vintage,variety,designation,release_price,rating,title,source,category,AVA,region,province,country,description
0,Griffin Creek,2011,Cabernet Franc,NaN,$40,91,Griffin Creek 2011 Cabernet Franc Rogue Valley,"Wine Spectator Magazine Issue Feb 28, 2015",None,Rogue Valley,Southern Oregon,Oregon,US,"Soft, ripe, generous and precise, this open-te..."
1,Quady North,2011,Arsenal,NaN,$40,86,Quady North 2011 Arsenal Rogue Valley,"Wine Spectator Magazine Issue Feb 02, 2017",None,Rogue Valley,Southern Oregon,Oregon,US,"A snappy red, if modest in scale, with blackbe..."
2,Del Rio,2011,Cabernet Sauvignon,NaN,$28,90,Del Rio 2011 Cabernet Sauvignon Rogue Valley,"Wine Spectator Magazine Issue Feb 28, 2014",None,Rogue Valley,Southern Oregon,Oregon,US,"A vibrant, medium-weight red, with ripe plum a..."
3,Griffin Creek,2011,Cabernet Sauvignon,NaN,$40,86,Griffin Creek 2011 Cabernet Sauvignon Rogue Va...,"Wine Spectator Magazine Issue Feb 28, 2015",None,Rogue Valley,Southern Oregon,Oregon,US,"Lean, spicy and firmly tannic, offering billow..."
4,Domaine Serene,2011,Chardonnay,Récolte Grand Cru,$125,94,Domaine Serene 2011 Chardonnay Dundee Hills Ré...,"Wine Spectator Magazine Issue Feb 28, 2015",None,Dundee Hills,Willamette Valley,Oregon,US,"Tangy, with vibrant acidity and expressive, ri..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,Knudsen,2015,Chardonnay,NaN,$45,90,Knudsen 2015 Chardonnay Dundee Hills,Wine Spectator Magazine Issue Web Only - 2017,None,Dundee Hills,Willamette Valley,Oregon,US,"Crisp and harmonious, with lemon blossom and a..."
1234,Ponzi,2015,Chardonnay,Reserve,$40,90,Ponzi 2015 Chardonnay Willamette Valley Reserve,"Wine Spectator Magazine Issue Jul 31, 2018",None,Willamette Valley,Willamette Valley,Oregon,US,"Graceful and polished, with expressive pear, s..."
1235,Raptor Ridge,2015,Chardonnay,Gran Moraine Vineyard,$45,90,Raptor Ridge 2015 Chardonnay Yamhill-Carlton D...,Wine Spectator Magazine Issue Web Only - 2017,None,Yamhill-Carlton District,Willamette Valley,Oregon,US,"Light and polished, showing pretty pear, pinea..."
1236,Roco,2015,Chardonnay,Marsh Estate Vineyards,$60,90,Roco 2015 Chardonnay Yamhill-Carlton District ...,Wine Spectator Magazine Issue Web Only - 2018,None,Yamhill-Carlton District,Willamette Valley,Oregon,US,"Floral and harmonious, with delicate tropical ..."


In [657]:
conn = engine.connect()

In [700]:
clean_spectator_df.to_sql(name='wine_reviews', con=engine, if_exists='append', index=True, index_label="review_id", chunksize=50)

In [702]:
[x for x in conn.execute("select count(*) from wine_reviews;")]

[(6611,)]

In [703]:
clean_spectator_df.to_csv("cleaned_wine_spectator_reviews.csv")